In [1]:
from enrichment import spatial_enrichment
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#csv_dir = r'C:\Users\IT-WL-annkl878\Box\CellClassification2023-01-12\Qupath_TMA5\output\LGG*.csv'
csv_dir = r'/media/fredrik/My Passport/Data/Panel 2/Tissue curation LGG TMA panel 2/Tissue curation TMA 8/output/*.csv'

# Get the list of all CSV files in the directory
csv_files = glob.glob(csv_dir)

# Create an empty DataFrame
df = pd.DataFrame()

# Loop through the CSV files and concatenate them into the DataFrame
for csv_file in csv_files:
    df = pd.concat([df, pd.read_csv(csv_file, delimiter='\t')], ignore_index=True)

In [3]:
#'Image'
#'Class'
#'Centroid X µm'
#'Centroid Y µm'   

In [4]:
cores = df.Image.unique()

#labels_combinations = []
#for partner1 in labels:
#    for partner2 in labels:
#        combination = partner1 + '_' + partner2 + '_' + str(nbh_radius)
#        labels_combinations.extend([combination])

#results_df = pd.DataFrame(columns = labels_combinations)
results_df = pd.DataFrame()

In [5]:
results_df = pd.DataFrame()

In [6]:
for c in range(0, len(cores)):
    core_df = df[df.Image == cores[c]]

    xy = core_df[['Centroid X µm', 'Centroid Y µm']].to_numpy()
    cell_classes = core_df['Class'].to_numpy()

    nbh_radius = 30

    #call spatial enrichment from Axel
    #enrich, labels = spatial_enrichment(xy, cell_classes, method='radius', r = nbh_radius)
    result = spatial_enrichment(xy, cell_classes, method='radius', r = nbh_radius)
    enrich = result["z_scores"]
    labels = result["labels"]

    # get lable combinations, not all cores might have cells of all celltypes
    labels_combinations = []
    for partner1 in labels:
        for partner2 in labels:
            combination = partner1 + '_' + partner2 + '_' + str(nbh_radius)
            labels_combinations.extend([combination])

    #linearize result to enter in results_table
    enrich_vector = np.reshape(enrich, np.shape(enrich)[0]* np.shape(enrich)[1])
    enrich_core_30 = dict(zip(labels_combinations, enrich_vector))

    nbh_radius = 50

    #call spatial enrichment from Axel
    #enrich, labels = spatial_enrichment(xy, cell_classes, method='radius', r = nbh_radius)
    result = spatial_enrichment(xy, cell_classes, method='radius', r = nbh_radius)
    enrich = result["z_scores"]
    labels = result["labels"]

    # get lable combinations, not all cores might have cells of all celltypes
    labels_combinations = []
    for partner1 in labels:
        for partner2 in labels:
            combination = partner1 + '_' + partner2 + '_' + str(nbh_radius)
            labels_combinations.extend([combination])

    #linearize result to enter in results_table
    enrich_vector = np.reshape(enrich, np.shape(enrich)[0]* np.shape(enrich)[1])
    enrich_core_50 = dict(zip(labels_combinations, enrich_vector))

    data_core = pd.DataFrame()
    data_core.loc[c, 'Name'] = cores[c]
    pd_enrich_30 = pd.DataFrame(enrich_core_30, index = [c])
    pd_enrich_50 = pd.DataFrame(enrich_core_50, index = [c])
    data_core = pd.concat([data_core, pd_enrich_30, pd_enrich_50], axis=1)

    # basic cell counts
    data_core['nCells'] = len(core_df)

    # todo: too complicated
    abundance_cells = core_df.groupby('Class')['Image'].count() 
    abundance_pd = pd.DataFrame(abundance_cells.to_dict(), index = [c])
    abundance_pd = abundance_pd.add_suffix('_abundance')

    fractions = core_df.groupby('Class')['Image'].count() / len(core_df)
    fractions_dict = fractions.to_dict()
    fractions_pd = pd.DataFrame(fractions_dict, index = [c])
    fractions_pd = fractions_pd.add_suffix('_fraction')

    data_core = pd.concat([data_core, abundance_pd, fractions_pd], axis=1)

    results_df = pd.concat([results_df, data_core], axis=0)

In [7]:
results_df.to_csv("output.csv")